# ML в Биологии
## 5. Введение в реккурентные нейронные сети

In [ ]:
!pip install scikit-image
!pip install lightning
!pip install tensorboard
!pip install mlflow

In [ ]:
import time
from tqdm.notebook import tqdm
from collections import defaultdict

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as sps
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import lightning.pytorch as pl
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.callbacks import ModelCheckpoint

import torch
from torch import nn
from torch.utils.data import Dataset, TensorDataset, \
                             DataLoader, RandomSampler, SequentialSampler

from IPython.display import clear_output
from pylab import rcParams

rcParams['figure.figsize'] = 15, 7
%matplotlib inline

sns.set(font_scale=1.3, palette='Set2')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Данные

## Биология

#### **Профиль биология**

### Загрузка данных
В этом задании вы будете работать с данными о распространении COVID-19, их можно скачать [здесь](https://www.kaggle.com/datasets/gpreda/coronavirus-2019ncov). Мы будем предказывать показатель смертности, потому что способы измерения выздоровевших и заболевших разнятся между регионами.

 В данных представлены следующие столбцы:

* `Country/Region` &mdash; страна или регион,

* `Province/State` &mdash; город или населенный пункт,
* `Latitude` &mdash; географическая широта,
* `Longitude` &mdash; географическая долгота,
* `Confirmed` &mdash; кол-во подтвержденных случаев заболевания,
* `Recovered` &mdash; кол-во подтвержденных случаев выздоровления,
* `Deaths` &mdash; кол-во смертей,
* `Date` &mdash; дата.

Выгрузим датасет.

In [ ]:
!unzip archive

In [ ]:
df = pd.read_csv('covid-19-all.csv', dtype={'Country/Region': str, 'Province/State': str}, low_memory=False)
df['Date'] = pd.to_datetime(df['Date'])
df.head()

Установим в качестве индекса дату.

In [ ]:
df.set_index('Date', inplace=True)
df.head()

Избавимся от None если они есть.

In [ ]:
df.dropna(inplace=True)
df.head()

Выберем страну и регион, где хотим предсказывать.

In [ ]:
country = 'Russia'
region = 'Moscow'

df_selected = df[(df['Country/Region'] == country) & (df['Province/State'] == region)]
df_selected.head()

Удалим лишние столбцы в выбранных данных.

In [ ]:
df_selected = df_selected[['Confirmed', 'Recovered', 'Deaths']]
df_selected.head()

Посмотрим на данные. Постройте графики заболевших, выздоровевших и количества смертей.

In [ ]:
# Построение графиков
plt.figure(figsize=(25, 15))

# График подтвержденных случаев
plt.subplot(3, 1, 1)
plt.plot(df_selected.index, df_selected['Confirmed'], label='Confirmed', color='blue')
plt.title('Confirmed Cases Over Time in Moscow, Russia')
plt.xlabel('Date')
plt.ylabel('Confirmed Cases')
plt.legend()
plt.grid(True)

# График выздоровевших
plt.subplot(3, 1, 2)
plt.plot(df_selected.index, df_selected['Recovered'], label='Recovered', color='green')
plt.title('Recovered Cases Over Time in Moscow, Russia')
plt.xlabel('Date')
plt.ylabel('Recovered Cases')
plt.legend()
plt.grid(True)

# График смертей
plt.subplot(3, 1, 3)
plt.plot(df_selected.index, df_selected['Deaths'], label='Deaths', color='red')
plt.title('Deaths Over Time in Moscow, Russia')
plt.xlabel('Date')
plt.ylabel('Deaths')
plt.legend()
plt.grid(True)

# Отображение графиков
plt.tight_layout()
plt.show()

# Модели и обучение

## Вспомогательные функции

Напишите класс датасета для данных в виде последовательности.

In [ ]:
class TSDataset(torch.utils.data.Dataset):
    def __init__(self, data, timesteps):
        self.data = data
        self.timesteps = timesteps

    def __len__(self):
        return self.data.shape[0] - self.timesteps

    def __getitem__(self, index):
        return self.data[index:index+self.timesteps], self.data[index+self.timesteps]

Здесь мы создаём функции для обучения, для вывода кривых обучения и для рекурсивного предсказания.

In [ ]:
def evaluate_ts_model(model, start_seq, test_data, scaler, return_all=False, device='cpu'):
    '''
    Функция для проверки качества модели на обучающем отрезке ряда.

    :param model: обучаемая модель,
    :param start_seq: обучающие данные для первого предсказания,
    :param test_data: тестовые данные.
    :param return_all: возвращать все предсказания или только для 1-го магазина

    :return: результаты предсказания.
    '''
    result = []
    model.train(False)
    input_tensor = torch.FloatTensor(start_seq).to(device).unsqueeze(0)

    with torch.no_grad():
        for i in range(len(test_data)):
            # делаем предсказание, а unsqueeze нужны, чтобы сделать размерность (1, 1, 1) вместо (1)

            logits = model(input_tensor[:, i:, :]).unsqueeze(0)#.unsqueeze(2)

            # присоединяем предсказанное значение к последовательности:
            #                        (1, timestep, 1) -> (1, 1, 1)   по оси 1
            input_tensor = torch.cat((input_tensor,        logits),       1    )

            # обратное преобразование к нормальным числам
            logits = scaler.inverse_transform(logits.cpu().numpy().squeeze(0))

            # результат сохраняем
            result.append(logits.squeeze())

    if return_all:
        return np.array(result)

    return np.array(result)

Создадим класс модели.

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_lstm_layers, use_pool=False):
        super(LSTM, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers  = num_lstm_layers
        self.input_size  = input_size

        self.lstm = nn.LSTM(input_size, hidden_size,
                            num_layers=num_lstm_layers,
                            batch_first=True,
                            dropout=0.2) # LSTM-модель с batch_first=True и dropout=0.2

        # Размерность пространства выхода последнего LSTM-слоя равна hidden_size.
        # Линейный слой нужен, чтобы преобразовать выход LSTM к нужному размеру output_size.
        self.fc = nn.Linear(in_features=hidden_size,
                            out_features=output_size) # добавьте линейный слой

        # Пуллинг
        self.pool = nn.AdaptiveAvgPool1d(1) # добавьте усредняющий все выходы пуллинг-слой
        self.use_pool = use_pool


    def forward(self, input_seq):

        # инициализируем начальные скрытые состояния
        h_0 = torch.zeros(self.num_layers, input_seq.size(0), self.hidden_size).to(device=input_seq.device)
        c_0 = torch.zeros(self.num_layers, input_seq.size(0), self.hidden_size).to(device=input_seq.device)

        out, (_, _) = self.lstm(input_seq, (h_0, c_0))

        if self.use_pool:
            # берем среднее от векторов для всей последовательности
            out_to_fc = self.pool(out.transpose(1, 2)).squeeze(-1)
        else:
            # берем последний выходной вектор
            out_to_fc = out[:, -1, :]

        return self.fc(out_to_fc)

Напишем функцию для визуализации результатов предсказания.

In [ ]:
def plot_results(y_to_train, y_to_test=None, y_forecast=None):
    """
        Функция для визуализации временного ряда и предсказания.

        Параметры:
            - y_to_train: pd.Series
                Временной ряд, на котором обучалась модель.
            - y_to_test: pd.Series
                Временной ряд, который предсказывает модель.
            - y_forecast: array
                Предсказания модели.
            - plot_conf_int: bool
                Надо ли строить предсказательного интервал.
            - left_bound: array
                Левая граница предсказательного интервала.
            - right_bound: array
                Правая граница предсказательного интервала.
    """
    plt.figure(figsize=(15, 5))
    plt.plot(np.arange(len(y_to_train)), y_to_train, label='train')

    if y_to_test is not None:
        plt.plot(np.arange(len(y_to_train), len(y_to_train) + len(y_to_test)), y_to_test,  label='test')
        if y_forecast is not None:
            plt.plot(np.arange(len(y_to_train), len(y_to_train) + len(y_to_test)), y_forecast, label='prediction')
    plt.legend()
    plt.show()

## Биология

Разделим выборку на тест и трейн.

In [ ]:
test_time = pd.Timestamp('2020-12-01')
train_bio = df_selected[df_selected.index < test_time]['Deaths'].values.reshape(-1, 1)
test_bio = df_selected[df_selected.index >= test_time]['Deaths'].values.reshape(-1, 1)
train_bio.shape, test_bio.shape

Посмотрим на то, что получилось.

In [ ]:
plot_results(train_bio, test_bio)

Преобразуем данные.

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
train_scaled = scaler.fit_transform(train_bio)

Зададим длину подпоследовательности, размер батча, девайс.

In [ ]:
timesteps = 5
batch_size = 32

Преобразуем к датасету с помощью нашего класса и сделаем генератор батчей.

In [ ]:
train_dataset = TSDataset(train_scaled, timesteps)
test_dataset = TSDataset(scaler.transform(test_bio), timesteps)

train_batch_gen = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=False)
test_batch_gen = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False)

Инициализируем модель.

In [ ]:
class TSModel(pl.LightningModule):
    def __init__(self, model, lr=5e-4):
        super().__init__()
        self.lr = lr
        self.criterion = nn.MSELoss()
        self.save_hyperparameters()
        self.model = model
        self.predictions = []

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.lr)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x_batch, y_batch = batch
        output = self.forward(x_batch)
        loss = self.criterion(output.squeeze(), y_batch.squeeze())
        self.log('train_loss', loss)
        return loss

    def training_step(self, batch, batch_idx):
        x_batch, y_batch = batch
        output = self.forward(x_batch)
        loss = self.criterion(output.squeeze(), y_batch.squeeze())
        self.log('val_loss', loss)
        return loss

    def transfer_batch_to_device(self, batch, device, dataloader_idx):
        x_batch, y_batch = batch
        x_batch = x_batch.type(torch.float32).to(device)
        y_batch = y_batch.type(torch.float32).to(device)
        return x_batch, y_batch

In [ ]:
lstm_1 = LSTM(
            input_size=1,
            output_size=1,
            hidden_size=50,
            num_lstm_layers=2,
            use_pool=False
            )

lstm_model_1 = TSModel(
                    model=lstm_1,
                    lr=5e-4,
                    )

In [ ]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor='val_loss', mode='max')
tb_logger = pl_loggers.TensorBoardLogger(save_dir="lightning_logs/lstm_1")

trainer = pl.Trainer(logger=tb_logger,
                     accelerator='gpu',
                     max_epochs=1000,
                     devices=1,
                     val_check_interval=5,
                     callbacks=[checkpoint_callback])

Создадим предсказания на тесте.

In [ ]:
trainer.fit(lstm_model_1, train_batch_gen, test_batch_gen)

Сравним предсказание с реальностью.

In [ ]:
lstm_model_1.cpu()
start_seq = train_scaled[-timesteps:]
test_pred = evaluate_ts_model(lstm_model_1, start_seq, scaler.transform(test_bio), scaler)
plot_results(train_bio, test_bio, test_pred)

In [ ]:
#%reload_ext tensorboard
#%tensorboard --logdir=lightning_logs/lstm_1

**Вывод:**

1. **Общая тенденция**:
   - График показывает, что модель в целом неплохо схватывает общую тенденцию изменений в количестве смертей. Однако предсказания значительно отклоняются от реальных данных в определенные моменты времени.

2. **Анализ предсказаний**:
   - Линия реальных значений, как правило, сохраняет более стабильный, линейный тренд, в то время как предсказания имеют логарифмический характер.
   - Предсказанные значения имеют значительные колебания, что может говорить о недостаточной адаптивности модели к изменяющимся условиям в течение времени.

3. **Периоды несоответствия**:
   - На графике можно заметить конкретные временные отрезки, где предсказания значительно расходятся с реальными данными. Это может указывать на влияние внешних факторов (например, всплески заболеваемости, изменения в политике, массовая вакцинация), которые модель не учла.

4. **Заключение**:
   - В общем, модель предоставляет полезные предсказания, но требует дальнейшего дообучения и адаптации для достижения большей точности.